<p style="align: center;"><img src="https://static.tildacdn.com/tild6636-3531-4239-b465-376364646465/Deep_Learning_School.png" width="400"></p>

# Домашнее задание. Обучение языковой модели с помощью LSTM 

Э
В этом задании Вам предстоит обучить языковую модель с помощью рекуррентной нейронной сети. В отличие от семинарского занятия, Вам необходимо будет работать с отдельными словами, а не буквами.


Установим модуль ```datasets```, чтобы нам проще было работать с данными.

Импорт необходимых библиотек

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import numpy as np
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from datasets import load_dataset
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.model_selection import train_test_split
import nltk

from collections import Counter
from typing import List

import seaborn
seaborn.set(palette='summer')

In [ ]:
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## Подготовка данных

Воспользуемся датасетом imdb. В нем хранятся отзывы о фильмах с сайта imdb. Загрузим данные с помощью функции ```load_dataset```

In [ ]:
# Загрузим датасет
dataset = load_dataset('imdb')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

### Препроцессинг данных и создание словаря (1 балл)

Далее вам необходмо самостоятельно произвести препроцессинг данных и получить словарь или же просто ```set``` строк. Что необходимо сделать:

1. Разделить отдельные тренировочные примеры на отдельные предложения с помощью функции ```sent_tokenize``` из бибилиотеки ```nltk```. Каждое отдельное предложение будет одним тренировочным примером.
2. Оставить только те предложения, в которых меньше ```word_threshold``` слов.
3. Посчитать частоту вхождения каждого слова в оставшихся предложениях. Для деления предлоения на отдельные слова удобно использовать функцию ```word_tokenize```.
4. Создать объект ```vocab``` класса ```set```, положить в него служебные токены '\<unk\>', '\<bos\>', '\<eos\>', '\<pad\>' и vocab_size самых частовстречающихся слов.   

In [ ]:
dataset = load_dataset('imdb')
nltk.download('punkt')
word_threshold = 32
sentences = []
for text in dataset['train']['text']:
    for sentence in sent_tokenize(text):
        words = word_tokenize(sentence.lower())
        if len(words) < word_threshold:
            sentences.append(words)




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
print("Всего предложений:", len(sentences))

Всего предложений: 198801


Посчитаем для каждого слова его встречаемость.

In [ ]:
from collections import Counter


word_counts = Counter(word for sentence in sentences for word in sentence)
print("Топ-10 самых частых слов:", word_counts.most_common(10))



Топ-10 самых частых слов: [('.', 170189), ('the', 151288), (',', 114121), ('a', 75273), ('and', 73159), ('of', 62796), ('to', 60494), ('is', 57468), ('it', 51862), ('i', 48989)]


Добавим в словарь ```vocab_size``` самых встречающихся слов.

In [ ]:

vocab_size = 40_000
vocab = {'<unk>', '<bos>', '<eos>', '<pad>'}
vocab.update(word for word, _ in word_counts.most_common(vocab_size))
word2ind = {word: i for i, word in enumerate(vocab)}
ind2word = {i: word for word, i in word2ind.items()}
print(f"Размер словаря: {len(vocab)}")

Размер словаря: 40004


In [ ]:
assert '<unk>' in vocab
assert '<bos>' in vocab
assert '<eos>' in vocab
assert '<pad>' in vocab
assert len(vocab) == vocab_size + 4

In [ ]:
print("Всего слов в словаре:", len(vocab))

Всего слов в словаре: 40004


### Подготовка датасета (1 балл)

Далее, как и в семинарском занятии, подготовим датасеты и даталоадеры.

В классе ```WordDataset``` вам необходимо реализовать метод ```__getitem__```, который будет возвращать сэмпл данных по входному idx, то есть список целых чисел (индексов слов).

Внутри этого метода необходимо добавить служебные токены начала и конца последовательности, а также токенизировать соответствующее предложение с помощью ```word_tokenize``` и сопоставить ему индексы из ```word2ind```.

In [ ]:
word2ind = {char: i for i, char in enumerate(vocab)}
ind2word = {i: char for char, i in word2ind.items()}

In [ ]:
from typing import List

class WordDataset:
    def __init__(self, sentences):
        self.data = sentences
        self.unk_id = word2ind['<unk>']
        self.bos_id = word2ind['<bos>']
        self.eos_id = word2ind['<eos>']
        self.pad_id = word2ind['<pad>']

    def __getitem__(self, idx: int) -> List[int]:
        sentence = self.data[idx]
        tokenized_sentence = [self.bos_id] + [word2ind.get(word, self.unk_id) for word in sentence] + [self.eos_id]

        return tokenized_sentence

    def __len__(self) -> int:
        return len(self.data)




In [ ]:
def collate_fn_with_padding(
    input_batch: List[List[int]], pad_id=word2ind['<pad>']) -> torch.Tensor:
    seq_lens = [len(x) for x in input_batch]
    max_seq_len = max(seq_lens)

    new_batch = []
    for sequence in input_batch:
        for _ in range(max_seq_len - len(sequence)):
            sequence.append(pad_id)
        new_batch.append(sequence)

    sequences = torch.LongTensor(new_batch).to(device)

    new_batch = {
        'input_ids': sequences[:,:-1],
        'target_ids': sequences[:,1:]
    }

    return new_batch

In [ ]:
train_sentences, eval_sentences = train_test_split(sentences, test_size=0.2)
eval_sentences, test_sentences = train_test_split(sentences, test_size=0.5)

train_dataset = WordDataset(train_sentences)
eval_dataset = WordDataset(eval_sentences)
test_dataset = WordDataset(test_sentences)

batch_size = 128

train_dataloader = DataLoader(
    train_dataset, collate_fn=collate_fn_with_padding, batch_size=batch_size)

eval_dataloader = DataLoader(
    eval_dataset, collate_fn=collate_fn_with_padding, batch_size=batch_size)

test_dataloader = DataLoader(
    test_dataset, collate_fn=collate_fn_with_padding, batch_size=batch_size)

## Обучение и архитектура модели

Вам необходимо на практике проверить, что влияет на качество языковых моделей. В этом задании нужно провести серию экспериментов с различными вариантами языковых моделей и сравнить различия в конечной перплексии на тестовом множестве.

Возмоэные идеи для экспериментов:

* Различные RNN-блоки, например, LSTM или GRU. Также можно добавить сразу несколько RNN блоков друг над другом с помощью аргумента num_layers. Вам поможет официальная документация [здесь](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html)
* Различные размеры скрытого состояния. Различное количество линейных слоев после RNN-блока. Различные функции активации.
* Добавление нормализаций в виде Dropout, BatchNorm или LayerNorm
* Различные аргументы для оптимизации, например, подбор оптимального learning rate или тип алгоритма оптимизации SGD, Adam, RMSProp и другие
* Любые другие идеи и подходы

После проведения экспериментов необходимо составить таблицу результатов, в которой описан каждый эксперимент и посчитана перплексия на тестовом множестве.

Учтите, что эксперименты, которые различаются, например, только размером скрытого состояния или количеством линейных слоев считаются, как один эксперимент.

Успехов!

### Функция evaluate (1 балл)

Заполните функцию ```evaluate```

In [ ]:
def evaluate(model, criterion, dataloader, device='cuda'):
    model.eval()
    perplexity = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids, target_ids = batch['input_ids'].to(device), batch['target_ids'].to(device)
            logits = model(input_ids)
            loss = criterion(logits.view(-1, logits.size(-1)), target_ids.flatten())
            perplexity.append(torch.exp(loss).item())

    return sum(perplexity) / len(perplexity)


### Train loop (1 балл)

Напишите функцию для обучения модели.

In [ ]:
def train_model(model, train_dataloader, eval_dataloader, num_epochs=3, lr=0.001, device='cuda'):
    model.to(device)
    criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    train_losses = []
    eval_perplexities = []

    for epoch in range(num_epochs):
        model.train()
        epoch_losses = []

        for batch in tqdm(train_dataloader, desc=f'Training epoch {epoch+1}:'):
            input_ids, target_ids = batch['input_ids'].to(device), batch['target_ids'].to(device)

            optimizer.zero_grad()
            logits = model(input_ids)
            loss = criterion(logits.view(-1, logits.size(-1)), target_ids.flatten())
            loss.backward()
            optimizer.step()

            epoch_losses.append(loss.item())

        avg_train_loss = sum(epoch_losses) / len(epoch_losses)
        train_losses.append(avg_train_loss)

        perplexity = evaluate(model, criterion, eval_dataloader, device)
        eval_perplexities.append(perplexity)

        print(f"Epoch {epoch+1}: Train Loss = {avg_train_loss:.4f}, Perplexity = {perplexity:.2f}")

    return train_losses, eval_perplexities


### Первый эксперимент (2 балла)

Определите архитектуру модели и обучите её.

In [ ]:
class LanguageModel(nn.Module): #базовая архитектура
    def __init__(self, vocab_size, embedding_dim=128, hidden_dim=256, num_layers=1, dropout=0.2):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, input_batch: torch.Tensor) -> torch.Tensor:
        embedded = self.embedding(input_batch)
        lstm_out, _ = self.lstm(embedded)
        lstm_out = self.dropout(lstm_out)
        output = self.fc(lstm_out)

        return output


In [ ]:
# Создаём модель
model_1 = LanguageModel(vocab_size=len(vocab), embedding_dim=128, hidden_dim=256, num_layers=1, dropout=0.2)

# Обучение модели
train_losses_1, eval_perplexities_1 = train_model(model_1, train_dataloader, eval_dataloader, num_epochs=10, lr=0.001)


Training epoch 1::   0%|          | 0/1243 [00:00<?, ?it/s]

Epoch 1: Train Loss = 5.5370, Perplexity = 152.21


Training epoch 2::   0%|          | 0/1243 [00:00<?, ?it/s]

Epoch 2: Train Loss = 4.9388, Perplexity = 113.79


Training epoch 3::   0%|          | 0/1243 [00:00<?, ?it/s]

Epoch 3: Train Loss = 4.7120, Perplexity = 94.81


Training epoch 4::   0%|          | 0/1243 [00:00<?, ?it/s]

Epoch 4: Train Loss = 4.5560, Perplexity = 83.34


Training epoch 5::   0%|          | 0/1243 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
!pip install --upgrade sympy


### Второй эксперимент (2 балла)

Попробуйте что-то поменять в модели или в пайплайне обучения, идеи для экспериментов можно подсмотреть выше.

In [ ]:
class LanguageModelV2(nn.Module): #2 LSTM слоя + Relu(вместо tanh)
    def __init__(self, vocab_size, embedding_dim=128, hidden_dim=256, num_layers=2, dropout=0.3):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim, vocab_size)
        self.activation = nn.ReLU()  # Добавили ReLU

    def forward(self, input_batch: torch.Tensor) -> torch.Tensor:
        embedded = self.embedding(input_batch)
        lstm_out, _ = self.lstm(embedded)
        lstm_out = self.dropout(lstm_out)
        output = self.fc(self.activation(lstm_out))  # Добавили ReLU перед линейным слоем

        return output


model_2 = LanguageModelV2(vocab_size=len(vocab), embedding_dim=128, hidden_dim=256, num_layers=2, dropout=0.3)
train_losses_2, eval_perplexities_2 = train_model(model_2, train_dataloader, eval_dataloader, num_epochs=4, lr=0.001)



Training epoch 1::   0%|          | 0/1243 [00:00<?, ?it/s]

Epoch 1: Train Loss = 6.3487, Perplexity = 326.64


Training epoch 2::   0%|          | 0/1243 [00:00<?, ?it/s]

Epoch 2: Train Loss = 5.4875, Perplexity = 174.46


Training epoch 3::   0%|          | 0/1243 [00:00<?, ?it/s]

Epoch 3: Train Loss = 5.1199, Perplexity = 139.30


Training epoch 4::   0%|          | 0/1243 [00:00<?, ?it/s]

Epoch 4: Train Loss = 4.9413, Perplexity = 120.86


У первой вариации перплексия ниже.

In [7]:
import nbformat

fn = 'pet_lm_dls.ipynb'
with open(fn, encoding='utf-8') as f:
    nb = nbformat.read(f, as_version=nbformat.NO_CONVERT)

if 'widgets' in nb['metadata']:
    del nb['metadata']['widgets']
    print('✅ Удалён metadata.widgets')
else:
    print('ℹ️ metadata.widgets не найден')

with open(fn, 'w', encoding='utf-8') as f:
    nbformat.write(nb, f)

print('💾 Файл сохранён:', fn)


✅ Удалён metadata.widgets
💾 Файл сохранён: pet_lm_dls.ipynb
